In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os

os.chdir("/content/drive/My Drive/Colab/assignment 2")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
myDataFrame2 = pd.read_csv('disabled.csv')
myDataFrame2.head()

,AgreementId,Version,Name,Region,Country,Peace Process,Peace Process Name,Stage,Signed Date,Agreement/Conflict Level,Conflict Nature,Agreement Status,Agreement Text
0,1923,2,Agreement between the two campaign teams regar...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2014-09-21,Intra,Government,Multiparty signed/agreed,Agreement between the Two Campaign Teams Regar...
1,1061,1,Afghanistan Compact Building on Success (Londo...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,SubPar,2006-02-01,InterIntra,Government,Multiparty signed/agreed,THE LONDON CONFERENCE ON AFGHANISTAN\r\n\r\n31...
2,908,1,Communiqué of the International Conference on ...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,Imp,2002-01-22,InterIntra,Government,Multiparty signed/agreed,Co-chairs' Summary of Conclusions\r\n\r\nThe I...
3,272,1,Agreement on Provisional Arrangements in Afgha...,Europe and Eurasia,Afghanistan,2,Afghanistan: 2000s Post-intervention process,SubPar,2001-12-05,Intra,Government,Multiparty signed/agreed,AGREEMENT ON PROVISIONAL ARRANGEMENTS IN AFGHA...
4,305,1,Angolan Government's Peace Plan,Africa (excl MENA),Angola,4,Angola: UNITA process,Imp,2002-03-13,Intra,Government,Multiparty signed/agreed,Angolan Government’s Peace Plan\r\n\r\n \r\n\r...


### Agreement Text
*   The last one to describe is the **Agreement Text** within this data. There are many stopwords need to be removed, and keywords need to be extracted.

In [4]:
myDataFrame5=myDataFrame2[['Agreement Text']]
myDataFrame5

,Agreement Text
0,Agreement between the Two Campaign Teams Regar...
1,THE LONDON CONFERENCE ON AFGHANISTAN\r\n\r\n31...
2,Co-chairs' Summary of Conclusions\r\n\r\nThe I...
3,AGREEMENT ON PROVISIONAL ARRANGEMENTS IN AFGHA...
4,Angolan Government’s Peace Plan\r\n\r\n \r\n\r...
...,...
116,AGREEMENT ON COMPREHENSIVE SOLUTIONS BETWEEN T...
117,Agreement between the Uganda Government and th...
118,National Dialogue Conference Outcomes Document...
119,APPENDIX F\r\nConflict Ceasefire Conditions\r\...


Create a new file for future use.

In [5]:
myDataFrame5.to_csv('agreement.csv')

Delete all blank lines in the file and save as another file.

In [6]:
f=open('agreement.csv')
g=open('re_agreement.csv','w')
try:
    while True:
        line=f.readline()
        if len(line)==0:
            break
        if line.count('\n')==len(line):
            continue
        g.write(line)
finally:
    f.close()
    g.close()

Clean the text with spacy method.

In [7]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    ldatokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            ldatokens.append('URL') 
        elif token.orth_.startswith('@'):
            ldatokens.append('SCREEN_NAME')
        else:
            ldatokens.append(token.lower_)
    return ldatokens

Lemmatization with wordnet method.

In [8]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def lemma1(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
from nltk.stem.wordnet import WordNetLemmatizer
def lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Use nltk to remove stopwords.

In [9]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
  def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens]
    return tokens

# lda

This is the data prepared for lda model, which can also first show some high frenquency words in the text.

In [11]:
import random
text_data = []
with open('re_agreement.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['participate', 'country', 'afghanistan', 'chair', 'australia', 'austria', 'bahrain', 'belgium', 'brazil', 'brunei', 'bulgaria', 'canada', 'china', 'czech', 'republic', 'denmark', 'egypt', 'finland', 'france', 'germany', 'greece', 'hungary', 'iceland', 'india', 'italy', 'japan', 'jordan', 'kazakhstan', 'korea', 'republic', 'kuwait', 'kyrgyztan', 'lithuania', 'luxembourg', 'malaysia', 'netherlands', 'zealand', 'norway', 'pakistan', 'poland', 'portugal', 'qatar', 'romania', 'russia', 'saudi', 'arabia', 'spain', 'sweden', 'switzerland', 'tajikistan', 'turkey', 'turkmenistan', 'unite', 'emirate', 'unite', 'kingdom', 'chair', 'unite', 'state', 'america', 'uzbekistan']
['official', 'transfer', 'power', 'interim', 'administration', 'shall', 'jurisdiction', 'printing', 'delivery', 'national', 'currency', 'special', 'drawing', 'right', 'international', 'financial', 'institution', 'interim', 'administration', 'shall', 'establish', 'assistance', 'unite', 'nation', 'central', 'afghanistan', 'regul

Use for building matrix.

In [12]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

5 topics for lda model.

In [13]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.021*"shall" + 0.011*"state" + 0.010*"right" + 0.010*"national"')
(1, '0.012*"party" + 0.011*"shall" + 0.008*"child" + 0.007*"national"')
(2, '0.024*"article" + 0.023*"shall" + 0.012*"national" + 0.009*"people"')
(3, '0.039*"shall" + 0.015*"national" + 0.014*"government" + 0.011*"public"')
(4, '0.013*"government" + 0.010*"education" + 0.010*"shall" + 0.009*"national"')


3 topics for lda model.

In [14]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.022*"national" + 0.022*"shall" + 0.016*"government" + 0.011*"state"')
(1, '0.025*"shall" + 0.020*"article" + 0.011*"president" + 0.008*"agreement"')
(2, '0.024*"shall" + 0.010*"include" + 0.009*"party" + 0.009*"right"')


10 topics for lda model.

In [15]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.044*"shall" + 0.011*"republic" + 0.010*"national" + 0.009*"following"')
(1, '0.034*"shall" + 0.014*"party" + 0.012*"agreement" + 0.011*"implementation"')
(2, '0.018*"shall" + 0.016*"rural" + 0.010*"education" + 0.009*"national"')
(3, '0.020*"security" + 0.019*"national" + 0.011*"police" + 0.008*"group"')
(4, '0.025*"shall" + 0.011*"south" + 0.010*"president" + 0.010*"parliament"')
(5, '0.022*"agreement" + 0.019*"government" + 0.017*"shall" + 0.014*"electoral"')
(6, '0.070*"article" + 0.023*"court" + 0.021*"shall" + 0.017*"commission"')
(7, '0.018*"public" + 0.018*"national" + 0.016*"shall" + 0.011*"council"')
(8, '0.032*"shall" + 0.021*"state" + 0.014*"national" + 0.013*"member"')
(9, '0.011*"right" + 0.011*"shall" + 0.010*"unite" + 0.009*"people"')


## Explore

### Use pyldavis to visualize different frequent words under different topics.

In [16]:
!pip install pyldavis

     |████████████████████████████████| 1.6MB 12.8MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=455a54a501cfa4c3a70d0640532b8aa0c6304102b8cc8fad6fc586acb70cb377
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


残疾人 5 主题

In [17]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

残疾人 3 主题

In [18]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


残疾人 10 主题

In [19]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
